In [ ]:
import tensorflow as tf
import keras
import fastdtw
import numpy as np
import matplotlib.pyplot as plt
import librosa
from librosa import display
import pandas as pd
import django

In [ ]:
for i in range(51,541):
    if(i<100):
        y,sr = librosa.core.load('ACCENT_CONVERSION_DATASET/cmu_us_awb_arctic/wav/arctic_a00' +str(i)+'.wav',sr=16000)
    else:
        y,sr = librosa.core.load('ACCENT_CONVERSION_DATASET/cmu_us_awb_arctic/wav/arctic_a0' +str(i)+'.wav',sr=16000)
    D = np.abs(librosa.core.stft(y,win_length=int(0.005*sr),n_fft=512))**2
    S = librosa.feature.melspectrogram(S=D,y=y,power=2.0)
    feats = librosa.feature.mfcc(S=librosa.power_to_db(S),n_mfcc=25,sr=sr,y=y)
    pd.DataFrame(feats).to_csv('cmu_us_awb_arctic/wav/arctic_a00' +str(i)+'.csv',header=None,index=None)

In [ ]:
from keras.models import Sequential
from keras.layers import Dropout, Dense
from sklearn.model_selection import train_test_split

In [ ]:
model = Sequential()
model.add(Dense(100),input_shape=(25,None),activation = 'tanh',use_bias=False)
model.add(Dense(100),activation = 'tanh',use_bias=False)
model.add(Dense(25),activation = 'tanh',use_bias=False)

model.compile(loss='mean_sqaured_error',optimizers='Adam',metrics=['accuracy'])

In [ ]:
X_train = 